# Depreciation to FERC 1 Connection

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import pudl
import pudl.constants as pc
import sqlalchemy as sa
import logging
import sys
import copy
import pathlib
import random
import warnings

sys.path.append("../")
import connect_deprish_to_eia
import make_plant_parts_eia
import connect_deprish_to_ferc1

In [3]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

### Prepare the inputs

In [4]:
file_path_mul = pathlib.Path().cwd().parent / 'master_unit_list.pkl.gz'
file_path_steam_ferc1 = pathlib.Path().cwd().parent / 'steam_ferc1.pkl.gz'
file_path_ferc1_eia = pathlib.Path().cwd().parent / 'ferc1_to_eia.pkl.gz'
file_path_deprish_eia = pathlib.Path().cwd().parent / 'deprish_to_eia.pkl.gz'

In [5]:
%%time
inputs = connect_deprish_to_ferc1.DeprishToFERC1Inputs(
    file_path_mul=file_path_mul,
    file_path_steam_ferc1=file_path_steam_ferc1,
    file_path_ferc1_eia=file_path_ferc1_eia,
    file_path_deprish_eia=file_path_deprish_eia
)

Reading the master unit list from /Users/christinagosnell/code/rmi-ferc1-eia/master_unit_list.pkl.gz
CPU times: user 2.11 s, sys: 511 ms, total: 2.63 s
Wall time: 2.83 s


### Generate the options and connections!

In [6]:
options_all_deprish_ferc1, options_deprish_ferc1, connects  = connect_deprish_to_ferc1.connect(inputs)

Connected plants:    102
Not connected:       38
Missing connections: 3
Portion of unique depreciation records:
    Matched:   83.09%
    No link:   10.63%
Connected:
one_ferc1_opt              117
same_true                   48
same_qualifiers_ferc1       13
same_qualifiers_deprish      7
same_diff_own                1
Name: connect_method, dtype: int64
Connection Levels:
NaN         211
beeg         57
smol         47
samezies     17
Name: level_deprish, dtype: int64


### Exploration of the non-connected options

In [7]:
# the missing ids
(options_deprish_ferc1[options_deprish_ferc1.record_id_eia_ferc1.notnull()].plant_id_pudl.unique())

array([248, 211, 426, 491, 105, 660, 410, 659, 165, 229, 196, 439, 737,
       657, 621, 110, 44, 509, 597, 508, 517, 596], dtype=object)

In [8]:
44 
# belle river. I'm pretty sure the deprish records are supposed
# to be associated with the two coal gens/units, but the steam table
# only inlcudes the oil and gas sections of the plant

105
# chesapeake. not positive, but it is possible that f1_steam_2011_12_186_0_4
# which is currently unmatched should be connected to the steam/coal
# portion of the plant

110
# chesterfield... fixed?noe

165
# cunningham... looks hopeless. May need overrides

165

In [9]:
# id to explore below
plant_id_pudl = 196

In [10]:
connects_deprish_eia = inputs.connects_deprish_eia
connects_deprish_eia[
    (connects_deprish_eia.plant_id_pudl == plant_id_pudl)
    #& (connects_deprish_eia.report_date.dt.year == 2016)
].sort_values('record_id_eia')

,record_id_eia,record_id_eia_fuzzy,plant_id_pudl,utility_name_ferc1,utility_id_pudl,state,plant_part_name_match,record_id_eia_name_match,plant_name,utility_id_ferc1,true_gran_name_match,plant_name_match,record_id_eia_override,record_id_eia_override10,record_id_eia_override2,record_id_eia_override3,record_id_eia_override4,record_id_eia_override5,record_id_eia_override6,record_id_eia_override7,record_id_eia_override8,record_id_eia_override9,report_date,plant_name_new,plant_part,report_year,ownership,plant_name_eia,plant_id_eia,generator_id,unit_id_pudl,prime_mover_code,energy_source_code_1,technology_description,ferc_acct_name,utility_id_eia,true_gran,appro_part_label,appro_record_id_eia
179,2442_2015_plant_owned_803,2442_1_2015_plant_gen_owned_803,196,Arizona Public Service Company,31,AZ,plant_gen,2442_1_2015_plant_gen_owned_803,four corners common,7,True,four corners 1,2442_2015_plant_owned_803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,Four Corners 2442,plant,2015,owned,Four Corners,2442,NaN,<NA>,ST,<NA>,Conventional Steam Coal,Steam,803,True,plant,2442_2015_plant_owned_803
178,2442_4_2015_plant_unit_owned_803,2442_4_2015_plant_unit_owned_803,196,Arizona Public Service Company,31,AZ,plant_gen,2442_4_2015_plant_gen_owned_803,four corners units 4-5,7,False,four corners 4,2442_4_2015_plant_unit_owned_803,2442_5_2015_plant_unit_owned_803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,Four Corners 4,plant_unit,2015,owned,Four Corners,2442,4,4,ST,SUB,Conventional Steam Coal,Steam,803,True,plant_unit,2442_4_2015_plant_unit_owned_803


In [11]:
connects[connects.plant_id_pudl == plant_id_pudl].sort_values('record_id_eia_deprish')[
        ['plant_part_deprish', 'plant_part_ferc1',
         'record_id_eia_deprish', 'record_id_eia_ferc1',
         'generator_id_deprish', 'generator_id_ferc1',
         'prime_mover_code_deprish', 'prime_mover_code_ferc1',
         'unit_id_pudl_deprish', 'unit_id_pudl_ferc1',
         'connect_method','count_ferc1'
        ]]

,plant_part_deprish,plant_part_ferc1,record_id_eia_deprish,record_id_eia_ferc1,generator_id_deprish,generator_id_ferc1,prime_mover_code_deprish,prime_mover_code_ferc1,unit_id_pudl_deprish,unit_id_pudl_ferc1,connect_method,count_ferc1


In [12]:
options_deprish_ferc1[options_deprish_ferc1.plant_id_pudl == plant_id_pudl][
    ['plant_part_deprish', 'plant_part_ferc1',
     'record_id_eia_deprish', 'record_id_eia_ferc1',
     #'generator_id_deprish', 'generator_id_ferc1',
     'prime_mover_code_deprish', 'prime_mover_code_ferc1',
     'unit_id_pudl_deprish', 'unit_id_pudl_ferc1',
     'plant_name','plant_name_new_deprish','plant_name_ferc1',
     'count_ferc1'
    ]]

,plant_part_deprish,plant_part_ferc1,record_id_eia_deprish,record_id_eia_ferc1,prime_mover_code_deprish,prime_mover_code_ferc1,unit_id_pudl_deprish,unit_id_pudl_ferc1,plant_name,plant_name_new_deprish,plant_name_ferc1,count_ferc1
255,plant_unit,plant_unit,2442_4_2015_plant_unit_owned_803,2442_5_2015_plant_unit_owned_803,ST,ST,4,5,four corners units 4-5,Four Corners 4,four corners 4,2
256,plant_unit,plant_unit,2442_4_2015_plant_unit_owned_803,2442_5_2015_plant_unit_owned_803,ST,ST,4,5,four corners units 4-5,Four Corners 4,four corners 5,2
257,plant,plant_unit,2442_2015_plant_owned_803,2442_5_2015_plant_unit_owned_803,ST,ST,<NA>,5,four corners common,Four Corners 2442,four corners 4,2
258,plant,plant_unit,2442_2015_plant_owned_803,2442_5_2015_plant_unit_owned_803,ST,ST,<NA>,5,four corners common,Four Corners 2442,four corners 5,2


In [13]:
ferc1_eia_df = inputs.connects_ferc1_eia
ferc1_eia_df[(ferc1_eia_df.plant_id_pudl == plant_id_pudl)
            & (ferc1_eia_df.report_date.dt.year.isin([2015]))
            ]

,record_id_ferc,record_id_eia,plant_name_new,plant_part,report_year,ownership,plant_name_eia,plant_id_eia,generator_id,unit_id_pudl,prime_mover_code,energy_source_code_1,technology_description,ferc_acct_name,utility_id_eia,true_gran,appro_part_label,appro_record_id_eia,plant_id_pudl,utility_id_pudl,utility_id_ferc1,plant_name_ferc1,plant_id_ferc1,capacity_factor,capacity_mw,net_generation_mwh,opex_fuel,fuel_cost_per_mwh,total_fuel_cost,total_mmbtu,construction_year,installation_year,fuel_type_code_pudl,plant_type,opex_production_total,fuel_cost_per_mmbtu,heat_rate_mmbtu_mwh,plant_id_report_year,report_date
3924,f1_steam_2015_12_176_1_1,2442_2015_plant_owned_24211,Four Corners 2442,plant,2015.0,owned,Four Corners,2442.0,NaN,<NA>,ST,<NA>,Conventional Steam Coal,Steam,24211,True,plant,2442_2015_plant_owned_24211,196,330,176,four corners,651,0.676853,114.53,679075.000,13209819.0,19.452666,1.320997e+07,6.624072e+06,1969,1970,coal,steam,20146976.0,1.994236,9.754551,196_2015,2015-01-01
4262,f1_steam_2015_12_7_1_5,2442_5_2015_plant_unit_owned_803,Four Corners 5,plant_unit,2015.0,owned,Four Corners,2442.0,5,5,ST,SUB,Conventional Steam Coal,Steam,803,True,plant_unit,2442_5_2015_plant_unit_owned_803,196,31,7,four corners 4,2117,0.748016,515.40,3377218.267,80951931.0,23.970003,8.095062e+07,3.337815e+07,1969,1970,coal,steam,120555552.0,2.425257,9.883328,196_2015,2015-01-01
4263,f1_steam_2015_12_7_2_1,2442_5_2015_plant_unit_owned_803,Four Corners 5,plant_unit,2015.0,owned,Four Corners,2442.0,5,5,ST,SUB,Conventional Steam Coal,Steam,803,True,plant_unit,2442_5_2015_plant_unit_owned_803,196,31,7,four corners 5,2118,0.696827,515.40,3146105.513,76278511.0,24.245376,7.627838e+07,3.107941e+07,1969,1970,coal,steam,115407981.0,2.454306,9.878692,196_2015,2015-01-01
9500,f1_steam_2015_12_147_0_2,NaN,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,196,274,147,four corners (1),427,0.769874,192.00,1294866.000,24646968.0,19.034377,2.278212e+07,1.267641e+07,1969,1970,coal,steam,38744128.0,1.797207,9.789744,196_2015,2015-01-01


In [14]:
plant_parts_eia = inputs.plant_parts_eia
plant_parts_eia[(plant_parts_eia.plant_id_pudl == plant_id_pudl)
                & (plant_parts_eia.report_year == 2015)
                #& (plant_parts_eia.prime_mover_code == 'ST')
                #& (plant_parts_eia.utility_id_pudl == 89)
                & (plant_parts_eia.plant_part.isin(['plant','plant_gen','plant_unit']))
                & (plant_parts_eia.utility_id_pudl ==31)
               ].sort_values('plant_part')

,record_id_eia,plant_id_eia,report_date,plant_part,generator_id,unit_id_pudl,prime_mover_code,energy_source_code_1,technology_description,ferc_acct_name,utility_id_eia,true_gran,appro_part_label,appro_record_id_eia,capacity_factor,capacity_mw,fraction_owned,fuel_cost_per_mmbtu,fuel_cost_per_mwh,fuel_type_code_pudl,heat_rate_mmbtu_mwh,installation_year,net_generation_mwh,operational_status,ownership,planned_retirement_date,plant_id_pudl,plant_name_eia,plant_name_new,total_fuel_cost,total_mmbtu,utility_id_pudl,utility_name_eia,report_year,plant_id_report_year
656031,2442_2015_plant_owned_803,2442,2015-01-01,plant,NaN,<NA>,ST,<NA>,Conventional Steam Coal,Steam,803,True,plant,2442_2015_plant_owned_803,0.439589,1664.206,0.770823,1.822403,17.780492,coal,9.756621,1970,6408518.76,NaN,owned,NaT,196,Four Corners,Four Corners 2442,1.139369e+08,9.923833e+07,31,Arizona Public Service Co,2015,196_2015
815341,2442_2015_plant_total_803,2442,2015-01-01,plant,NaN,<NA>,ST,<NA>,Conventional Steam Coal,Steam,803,True,plant,2442_2015_plant_total_803,0.511639,2269.600,1.000000,1.822403,17.780492,coal,9.756621,1970,10172252.00,NaN,total,NaT,196,Four Corners,Four Corners 2442,1.808522e+08,9.923833e+07,31,Arizona Public Service Co,2015,196_2015
480219,2442_1_2015_plant_gen_owned_803,2442,2015-01-01,plant_gen,1,<NA>,ST,BIT,Conventional Steam Coal,Steam,803,True,plant_gen,2442_1_2015_plant_gen_owned_803,0.000000,190.000,1.000000,NaN,NaN,coal,NaN,1963,0.00,NaN,owned,NaT,196,Four Corners,Four Corners 1,0.000000e+00,0.000000e+00,31,Arizona Public Service Co,2015,196_2015
480220,2442_2_2015_plant_gen_owned_803,2442,2015-01-01,plant_gen,2,<NA>,ST,BIT,Conventional Steam Coal,Steam,803,True,plant_gen,2442_2_2015_plant_gen_owned_803,0.000000,190.000,1.000000,NaN,NaN,coal,NaN,1963,0.00,NaN,owned,NaT,196,Four Corners,Four Corners 2,0.000000e+00,0.000000e+00,31,Arizona Public Service Co,2015,196_2015
480221,2442_3_2015_plant_gen_owned_803,2442,2015-01-01,plant_gen,3,<NA>,ST,BIT,Conventional Steam Coal,Steam,803,True,plant_gen,2442_3_2015_plant_gen_owned_803,0.000000,253.400,1.000000,NaN,NaN,coal,NaN,1964,0.00,NaN,owned,NaT,196,Four Corners,Four Corners 3,0.000000e+00,0.000000e+00,31,Arizona Public Service Co,2015,196_2015
480222,2442_4_2015_plant_gen_owned_803,2442,2015-01-01,plant_gen,4,4,ST,SUB,Conventional Steam Coal,Steam,803,False,plant_unit,2442_4_2015_plant_unit_owned_803,0.735011,515.403,0.630000,1.822403,17.737896,coal,9.733248,1969,3318521.85,NaN,owned,NaT,196,Four Corners,Four Corners 4,5.886360e+07,5.126983e+07,31,Arizona Public Service Co,2015,196_2015
480223,2442_5_2015_plant_gen_owned_803,2442,2015-01-01,plant_gen,5,5,ST,SUB,Conventional Steam Coal,Steam,803,False,plant_unit,2442_5_2015_plant_unit_owned_803,0.684395,515.403,0.630000,1.822403,17.823088,coal,9.779995,1970,3089996.91,NaN,owned,NaT,196,Four Corners,Four Corners 5,5.507329e+07,4.796850e+07,31,Arizona Public Service Co,2015,196_2015
812856,2442_4_2015_plant_gen_total_803,2442,2015-01-01,plant_gen,4,4,ST,SUB,Conventional Steam Coal,Steam,803,False,plant_unit,2442_4_2015_plant_unit_total_803,0.735011,818.100,1.000000,1.822403,17.737896,coal,9.733248,1969,5267495.00,existing,total,NaT,196,Four Corners,Four Corners 4,9.343428e+07,5.126983e+07,31,Arizona Public Service Co,2015,196_2015
812861,2442_5_2015_plant_gen_total_803,2442,2015-01-01,plant_gen,5,5,ST,SUB,Conventional Steam Coal,Steam,803,False,plant_unit,2442_5_2015_plant_unit_total_803,0.684395,818.100,1.000000,1.822403,17.823088,coal,9.779995,1970,4904757.00,existing,total,NaT,196,Four Corners,Four Corners 5,8.741792e+07,4.796850e+07,31,Arizona Public Service Co,2015,196_2015
658111,2442_4_2015_plant_unit_owned_803,2442,2015-01-01,plant_unit,4,4,ST,SUB,Conventional Steam Coal,Steam,803,True,plant_unit,2442_4_2015_plant_unit_owned_803,0.735011,515.403,0.630000,1.822403,17.737896,coal,9.733248,1969,3318521.85,NaN,owned,NaT,196,Four Corners,Four Corners 4,5.886360e+07,5.126983e+07,31,Arizona Public Service Co,2015,196_2015


In [15]:
def get_connects_diff_level_same_qualifiers(options_deprish_ferc):
    df = pd.DataFrame()
    for part_name in inputs.plant_parts_ordered:
        part_df = options_deprish_ferc1[options_deprish_ferc1.plant_part_ferc1 == part_name]
        # add the slice of the df for this part if the id columns for both 
        df = df.append(part_df[
            part_df[f"{inputs.parts_to_ids[part_name]}_deprish"] 
            == part_df[f"{inputs.parts_to_ids[part_name]}_ferc1"]])
    df = df.assign(connect_method="diff_level_same_qualifiers")
    return df